In [ ]:
# start here by entering a ticker symbol
ticker = "XXII"
# number of weeks to look back
weeks = 13
# and watch the magic happen below...
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from empyrical import roll_max_drawdown, alpha, beta
from sklearn.preprocessing import MinMaxScaler
from src.asset_selector import AssetSelector
from src.indicators import Indicators
from util import time_formatter, df2csv
from src.predictor import Predictor
from sklearn.cluster import KMeans
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
import pandas as pd
import numpy as np
import configparser
import requests
import json
import time
import sys
import os
backdate = time_formatter(time.time() - (604800 * weeks))
config = configparser.ConfigParser()
try:
    config.read(os.path.relpath("config.ini"))
except FileExistsError as e:
    print("FileExistsError: {}".format(e))
    sys.exit(1)
alpaca_api = tradeapi.REST(
    base_url    = config["alpaca"]["APCA_API_BASE_URL"],
    key_id      = config["alpaca"]["APCA_API_KEY_ID"],
    secret_key  = config["alpaca"]["APCA_API_SECRET_KEY"],
    api_version = config["alpaca"]["VERSION"]
)
trading_account = alpaca_api.get_account()

selector            = AssetSelector(alpaca_api, edgar_token=None)
indicators          = Indicators(alpaca_api)
scaler              = MinMaxScaler()
day_bars            = selector.get_barset(ticker, "day", backdate)
day_bars            = selector.extract_bar_data(day_bars, ticker)
minute_bars         = selector.get_barset(ticker, "minute", backdate)
minute_bars         = selector.extract_bar_data(minute_bars, ticker)
day_spy_bars        = selector.get_barset("SPY", "1D", backdate)
day_spy_bars        = selector.extract_bar_data(day_spy_bars, "SPY")
minute_spy_bars     = selector.get_barset("SPY", "minute", backdate)
minute_spy_bars     = selector.extract_bar_data(minute_spy_bars, "SPY")
ticker_cluster      = indicators.cluster_prep(ticker)
ticker_cluster["spy_day_close"] = day_spy_bars["close"]

In [ ]:
# testing csv output function
df2csv(day_bars, ticker)

In [ ]:
df2csv(day_spy_bars, "SPY")

In [ ]:
df2csv(minute_bars, ticker)

In [ ]:
ticker_cluster.iloc[-1]

# Time to start using the most recent values of these indicators to make some trade decisions!!!!

In [ ]:
# Normalizing, reshaping and scaling the dataframe 
# TODO: factor this out into a util function
open = np.array(ticker_cluster["open"])
open = open.reshape(-1,1)
ticker_cluster["open"] = scaler.fit_transform(open)

high = np.array(ticker_cluster["high"])
high = high.reshape(-1,1)
ticker_cluster["high"] = scaler.fit_transform(high)

low = np.array(ticker_cluster["low"])
low = low.reshape(-1,1)
ticker_cluster["low"] = scaler.fit_transform(low)

close = np.array(ticker_cluster["close"])
close = close.reshape(-1,1)
ticker_cluster["close"] = scaler.fit_transform(close)

volume = np.array(ticker_cluster["volume"])
volume = volume.reshape(-1,1)
ticker_cluster["volume"] = scaler.fit_transform(volume)

macd = np.array(ticker_cluster["macd"])
macd = macd.reshape(-1,1)
ticker_cluster["macd"] = scaler.fit_transform(macd)

signal = np.array(ticker_cluster["signal"])
signal = signal.reshape(-1,1)
ticker_cluster["signal"] = scaler.fit_transform(signal)

vzo = np.array(ticker_cluster["vzo"])
vzo = vzo.reshape(-1, 1)
ticker_cluster["vzo"] = scaler.fit_transform(vzo)

mfi = np.array(ticker_cluster["mfi"])
mfi = mfi.reshape(-1, 1)
ticker_cluster["mfi"] = scaler.fit_transform(mfi)

vzo = np.array(ticker_cluster["vzo"])
vzo = vzo.reshape(-1, 1)
ticker_cluster["vzo"] = scaler.fit_transform(vzo)

dmp = np.array(ticker_cluster["DMp"])
dmp = dmp.reshape(-1, 1)
ticker_cluster["DMp"] = scaler.fit_transform(dmp)

dmm = np.array(ticker_cluster["DMm"])
dmm = dmm.reshape(-1, 1)
ticker_cluster["DMm"] = scaler.fit_transform(dmm)

bb_up = np.array(ticker_cluster["bb_up"])
bb_up = bb_up.reshape(-1, 1)
ticker_cluster["bb_up"] = scaler.fit_transform(bb_up)

bb_mid = np.array(ticker_cluster["bb_mid"])
bb_mid = bb_mid.reshape(-1, 1)
ticker_cluster["bb_mid"] = scaler.fit_transform(bb_mid)

bb_low = np.array(ticker_cluster["bb_low"])
bb_low = bb_low.reshape(-1, 1)
ticker_cluster["bb_low"] = scaler.fit_transform(bb_low)

apz_u = np.array(ticker_cluster["apz_u"])
apz_u = apz_u.reshape(-1, 1)
ticker_cluster["apz_u"] = scaler.fit_transform(apz_u)

apz_l = np.array(ticker_cluster["apz_l"])
apz_l = apz_l.reshape(-1, 1)
ticker_cluster["apz_l"] = scaler.fit_transform(apz_l)

stoch = np.array(ticker_cluster["stoch"])
stoch = stoch.reshape(-1, 1)
ticker_cluster["stoch"] = scaler.fit_transform(stoch)

spy_day_close = np.array(ticker_cluster["spy_day_close"])
spy_day_close = spy_day_close.reshape(-1,1)
ticker_cluster["spy_day_close"] = scaler.fit_transform(spy_day_close)

# Exploratory graphs (using day bars)

In [ ]:
fig = plt.figure(figsize=(10,4), dpi=100)
fig.suptitle("Open, high, low, close for {} vs SPY".format(ticker), fontsize=10)
plt.plot(ticker_cluster["open"], color="orange")
plt.plot(ticker_cluster["high"], color="blue")
plt.plot(ticker_cluster["low"], color="brown")
plt.plot(ticker_cluster["close"], color="green")
plt.plot(ticker_cluster["spy_day_close"], color="black")

fig = plt.figure(figsize=(10,4), dpi=100)
fig.suptitle("Close MACD vs signal for {}".format(ticker), fontsize=10)
plt.plot(ticker_cluster["close"], color="green")
plt.plot(ticker_cluster["macd"], color="purple")
plt.plot(ticker_cluster["signal"], color="pink")

fig = plt.figure(figsize=(10,4), dpi=100)
fig.suptitle("Close price vs mfi for {}".format(ticker), fontsize=10)
plt.plot(ticker_cluster["close"], color="green")
plt.plot(ticker_cluster["mfi"], color="blue")

fig = plt.figure(figsize=(10,4), dpi=100)
fig.suptitle("Adaptive price zone for {}".format(ticker), fontsize=10)
plt.plot(ticker_cluster["close"], color="green")
plt.plot(ticker_cluster["apz_u"], color="grey")
plt.plot(ticker_cluster["apz_l"], color="grey")

fig = plt.figure(figsize=(10,4), dpi=100)
fig.suptitle("Stochastic oscillator vs close price for {}".format(ticker), fontsize=10)
plt.plot(ticker_cluster["close"], color="green")
plt.plot(ticker_cluster["stoch"], color="orange")

fig = plt.figure(figsize=(10,4), dpi=100)
fig.suptitle("Volume zone oscillator for {}".format(ticker), fontsize=10)
plt.plot(ticker_cluster["close"], color="green")
plt.plot(ticker_cluster["vzo"], color="orange")

In [ ]:
minute_close = minute_bars["close"]
minute_vwap = indicators.get_vwap(minute_bars)
minute_macd = indicators.get_macd(minute_bars)

min_close = np.array(minute_close)
min_close = min_close.reshape(-1, 1)
min_close = scaler.fit_transform(min_close)

min_vwap = np.array(minute_vwap)
min_vwap = min_vwap.reshape(-1, 1)
min_vwap = scaler.fit_transform(min_vwap)

min_macd = np.array(minute_macd["MACD"])
min_macd = min_macd.reshape(-1, 1)
min_macd = scaler.fit_transform(min_macd)

min_signal = np.array(minute_macd["SIGNAL"])
min_signal = min_signal.reshape(-1, 1)
min_signal = scaler.fit_transform(min_signal)

In [ ]:
fig = plt.figure(figsize=(10,4), dpi=100)
fig.suptitle("VWAP vs close for {}".format(ticker), fontsize=10)
plt.plot(min_vwap[-100:], color="blue")
plt.plot(min_close[-100:], color="orange")

fig = plt.figure(figsize=(10,4), dpi=100)
fig.suptitle("Close vs MACD + signal -- minute bars (last 50 periods) for {}".format(ticker), fontsize=10)
plt.plot(min_close[-100:], color="orange")
plt.plot(min_macd[-100:], color="red")
plt.plot(min_signal[-100:], color="green")

fig = plt.figure(figsize=(10,4), dpi=100)
fig.suptitle("Close vs VWAP -- minute bars (last 50 periods) for {}".format(ticker), fontsize=10)
plt.plot(min_close[-100:], color="orange")
plt.plot(min_vwap[-100:], color="blue")

In [ ]:
# Heatmap time -- had to create a separate dataframe sans date index column for this 
nd = pd.DataFrame()
nd["close"]     = ticker_cluster["close"]
nd["macd"]      = ticker_cluster["open"]
nd["signal"]    = ticker_cluster["open"]
nd["mfi"]       = ticker_cluster["open"]
nd["vzo"]       = ticker_cluster["open"]
nd["volume"]    = ticker_cluster["open"]
nd["bb_up"]     = ticker_cluster["bb_up"]
nd["bb_low"]    = ticker_cluster["bb_low"]
heatmap = go.Heatmap(
    z=nd.iloc[-50:].pct_change().corr(method='pearson').values,
    x=nd.columns,
    y=nd.columns,
    colorbar=dict(title='Pearson Coefficient'),
)
layout = go.Layout(title="Pearson correlation of indicators")
heatmap['zmax'] = 1.0
heatmap['zmin'] = -1.0
fig = go.Figure(data=[heatmap], layout=layout)
py.iplot(fig)

In [ ]:
fig = plt.figure(figsize=(10,4), dpi=100)
fig.suptitle("Minute bar closing price for {}".format(ticker), fontsize=10)
plt.plot(minute_bars["close"].iloc[-600:])

In [ ]:
fig = plt.figure(figsize=(10,4), dpi=100)
fig.suptitle("Minute bar closing price for {}".format(ticker), fontsize=10)
plt.plot(minute_bars["close"].iloc[-60:], color="red")

mmacd = indicators.get_macd(minute_bars.iloc[-600:])

fig = plt.figure(figsize=(10,4), dpi=100)
fig.suptitle("Minute bar MACD/signal for {}".format(ticker), fontsize=10)
plt.plot(mmacd["MACD"][-60:], color="orange")
plt.plot(mmacd["SIGNAL"][-60:], color="green")